In [1]:
import re
import numpy as np

In [2]:
def remove_special_chars(text):
    remove_tag = r'。|◎|■|▼|◆|▽|\.|●|①|②|③|;|,|、|★|☆|＊|◇|※|／／｜～' \
                 r'|\*|…|│|\?|（※）|「|」|『|』|\u3000' \
                 r'!|♪|！|％|%|\?|\／|\/|&|>|<|\|【|【|】|＜|＞' \
                 r'|\[|\]|\r|\n|\t|\'|_|e.g|\{|\}|\#|\@|\$'
    text = re.sub(remove_tag, ' ', text)
    text = re.sub('\s\s+', ' ', text.strip())
    return text

In [3]:
def split2sentences(para):
    paras = []
    # para = remove_numbers(para)
    paras = remove_special_chars(para).split(" ")
    return paras

In [4]:
match_pattern = re.compile(r'(転職(者|社|回)*(数|で))|(次[0-9]+(社|者)目)|(転職(者|社|回)*(数|で)[0-9]+((回以上)|(社|者)*))|(数*[0-9]+((社以内)|((社|者)*経験.{0,1}[迄|(まで)|(のみ)]*)))|(数*[0-9]+((社以内)|((社|者)(経験)*.{0,3}[迄|(まで)|(のみ)])))')

In [5]:
def extract_changeover_age_pattern(text):
    patterns = []
    sentences = split2sentences(text)
    for sentence in sentences:
        match = re.search(match_pattern, sentence)
        if match is not None:
            patterns.append(match.string)
    return patterns

In [6]:
def extract_number(text):
    numbers = []
    patterns = extract_changeover_age_pattern(text)
    for pattern in patterns:
        match = analyze_pattern(pattern)
        if len(match) != 0:
            numbers.append(match)
    return numbers

In [7]:
def analyze_pattern(pattern):
    match = re.findall(number_pattenr,pattern)
    match = list(map(lambda item: int(item), match))
    match = sorted(match,reverse=False)
    contain_changeover = '社' in pattern or '者' in pattern or '回' in pattern
    contain_age = '代' in pattern or '歳' in pattern
    if contain_changeover and not contain_age:
        if '社' in pattern or '者' in pattern:
            return (match[0]-1,0,200)
        else:
            return (match[0],0,200)
    if not contain_changeover and contain_age:
        if '代' in pattern:
            return (200,match[0],match[0] + 9)
        else:
            return (200,0,match[0])
    if contain_changeover and contain_age:
        if '社' in pattern or '者' in pattern:
            if '代' in pattern:
                return(match[0]-1,match[1],match[1]+9)
            elif '歳' in pattern:
                return(match[0]-1,0,match[1])
        elif '回' in pattern:
            if '代' in pattern:
                return(match[0],match[1],match[1]+9)
            elif '歳' in pattern:
                return(match[0],0,match[1])
        

In [36]:
sample_text = '転職回数厳守!…20代:2社経験、30代:3社経験まで';

In [ ]:
extract_changeover_age_pattern(sample_text)

In [9]:
number_pattern = re.compile(r'\d+')

def extract_change_over(text):
    patterns = extract_changeover_age_pattern(text)
    result = []
    
    for pattern in patterns:
        match = list(re.finditer(number_pattern, pattern))
        
        age_from = 0 
        age_to = 200
        time_change = 0
        if (len(match) == 0):
            continue
        
        if (int(match[0].group()) > 10):
            is_timechange_previous = False
        else:
            is_timechange_previous = True
        
        for x in match:
            number = int(x.group())
            position = x.end()
            if (position == len(pattern)):
                next_char = "x"
            else:
                next_char = pattern[position]
                
            if (number > 10 and is_timechange_previous == True):
                is_same = False
                if (age_to == 200 and age_from != 0):
                    age_to = age_from
                    age_from = 0
                    
                for idx, temp in enumerate(result):
                    if (temp[0] == age_from and temp[1] == age_to):
#                         result[idx] = (temp[0], temp[1], time_change)
#                         is_same = True
                        break
                if (is_same == False):
                    result.append((age_from, age_to, time_change))
                    
                age_from = 0 
                age_to = 200
                
            if (number < 10):
                time_change = number
                is_timechange_previous = True
            else:
                if (next_char == "代"):
                    if (age_from == 0):
                        age_from = number
                        age_to = (int(number/10) + 1) * 10 - 1
                    else:
                        age_to = number
                else:
                    if (age_from == 0):
                        age_from = number
                    else:
                        age_to = number 
                is_timechange_previous = False
        
        if (is_timechange_previous == True):
            is_same = False
            if (age_to == 200 and age_from != 0):
                    age_to = age_from
                    age_from = 0
            for idx, temp in enumerate(result):
                if (temp[0] == age_from and temp[1] == age_to):
#                     result[idx] = (temp[0], temp[1], time_change)
#                     is_same = True
                    break
            if (is_same == False):
                result.append((age_from, age_to, time_change))
                
    return result

In [37]:
temp = extract_change_over(sample_text)

In [38]:
print(temp)

[(20, 29, 2), (30, 39, 3)]


In [54]:
def extract_toeic(text):
    toeic_pattern = re.compile(r'TOIEC\d+|TOEIC\d+')
    sentences = split2sentences(text)
    for sentence in sentences:
        match = re.search(toeic_pattern, sentence)
        if match is not None:
            print(re.smatch.group())

In [57]:
sample_text = '必須要件】■物流会社あるいは商社等での輸出業務経験5年以上■英語力(TOEIC600点以上)'

In [58]:
extract_toeic(sample_text)

TOEIC600
